In [59]:
import numpy as np
import time
from sklearn.datasets import fetch_20newsgroups 

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 

from sklearn.linear_model import SGDClassifier 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV


## Creating test + train sets

In [4]:
categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']

# currently looking at all categories 
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, remove=('headers', 'footers', 'quotes'))
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, remove=('headers', 'footers', 'quotes'))


In [6]:
model_names = ['SVM', 'Logistic Regression', 'AdaBoost', 'Decision Tree', 'Random Forest']

# just need to add randomforest 
models = [
    LinearSVC(random_state=0),
    LogisticRegression(random_state=0),
    AdaBoostClassifier(n_estimators=50, learning_rate=1,random_state=0),
    DecisionTreeClassifier(random_state=0),
    RandomForestClassifier(max_depth=2, random_state=0)
]

parameter_grids = [
    [ # Parameter Grid for Linear SVC.
#         { # l1 Case: Dual False, Squared_hinge
#             'vect__ngram_range': [(1, 1), (1, 2)],
#             'tfidf__use_idf': [True, False],
#             'clf__loss': ['squared_hinge'],
#             'clf__penalty': ['l1'],
#             'clf__C': range(1,100,10),
#             'clf__tol': [1e-2, 1e-4, 1e-9],
#             'clf__dual': [False]
#         },
#         { # l2 Hinge Case: Dual True
#             'vect__ngram_range': [(1, 1), (1, 2)],
#             'tfidf__use_idf': [True, False],
#             'clf__loss': ['hinge'],
#             'clf__penalty': ['l2'],
#             'clf__C': range(1,100,10),
#             'clf__tol': [1e-2, 1e-4, 1e-9],
#             'clf__dual': [True]
#         },
#         { # l2 Square Hinged Case
#             'vect__ngram_range': [(1, 1), (1, 2)],
#             'tfidf__use_idf': [True, False],
#             'clf__loss': ['squared_hinge'],
#             'clf__penalty': ['l2'],
#             'clf__C': range(1,100,10),
#             'clf__tol': [1e-2, 1e-4, 1e-9],
#             'clf__dual': [True,False]
#         },
        {
            'vect__ngram_range': [(1,1), (1,2)],
            'tfidf__use_idf': (True, False),
            'vect__stop_words': ('english', None)
        }
    ],
    [
        {
            'vect__ngram_range': [(1, 1), (1, 2)],
            'tfidf__use_idf': (True, False),
            'clf__penalty': ('l2', 'l1')
        }
    ],
    [
        {
            'vect__ngram_range': [(1, 1), (1, 2)],
            'tfidf__use_idf': (True, False),
            'clf__learning_rate': (1, 2, 0.5),
            'clf__n_estimators': (10, 50, 100)
        }
    ],
    [
        {
            'vect__ngram_range': [(1, 1), (1, 2)],
            'tfidf__use_idf': (True, False),
            'clf__criterion': ('gini', 'entropy'),
        },
    ],
    [
        {
            'vect__ngram_range': [(1, 1), (1, 2)],
            'tfidf__use_idf': (True, False),
            'clf__criterion': ('gini', 'entropy'),
        }
    ]
]

In [29]:
def runGridSearchCV(parameter_grids, models, model_names):
    start = time.time()
    i = 0
    best_scores = []
    best_params = []
    for model in models: 
        print("Currently training model: ", model_names[i])

        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', model)])

        _  = text_clf.fit(twenty_train.data, twenty_train.target)    

        gs_clf = GridSearchCV(text_clf, parameter_grids[i], n_jobs=-1, cv=3,error_score=0.0)
        gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

        best_scores.append(gs_clf.best_score_)
        best_params.append(gs_clf.best_params_)

        print("Time taken: ", time.time()-start)
        print("Best score : ", gs_clf.best_score_)
        print("Best params: ", gs_clf.best_params_)
        print("\n\n")

        i+=1

In [53]:
def runRandomizedSearchCV(parameter_grids, models, model_names, num_iters=100):
    start = time.time()
    i = 0
    best_scores = []
    best_params = []
    for model in models: 
        print("Currently training model: ", model_names[i])

        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', model)])

        _  = text_clf.fit(twenty_train.data, twenty_train.target)    

        rs_clf = RandomizedSearchCV(text_clf, parameter_grids[i], n_jobs=-1, cv=3,error_score=0.0,n_iter = num_iters, verbose = 2, random_state=0)
        rs_clf = rs_clf.fit(twenty_train.data, twenty_train.target)

        best_scores.append(rs_clf.best_score_)
        best_params.append(rs_clf.best_params_)

        print("Time taken: ", time.time()-start)
        print("Best score : ", rs_clf.best_score_)
        print("Best params: ", rs_clf.best_params_)
        print("\n\n")

        i+=1

In [21]:
runGridSearchCV(parameter_grids, models, model_names)

TypeError: runGridSearchCV() takes 2 positional arguments but 3 were given

## Finding Hyperparameters For Each Model

### Linear SVC Parameters

In [62]:
linear_svc_rs = [
    {
        'vect__ngram_range': [(1,1),(1,2)],
        'tfidf__use_idf': [True,False],
        'clf__C': [0.01, 0.05, 0.1, 0.5, 1.0, 10, 20 , 30, 40, 50, 60, 70, 80, 90, 100],
        'clf__penalty': ['l2','l1'],
        'clf__loss': ['hinge','squared_hinge'],
    }
]

In [36]:
# Narrowed from randomized search
linear_svc_gs = [
    {
        'vect__ngram_range': [(1,1),(1,2)],
        'tfidf__use_idf': [True,False],
        'clf__C': [0.01, 1.0, 100.],
        'clf__penalty': ['l2','l1'],
        'clf__loss': ['hinge','squared_hinge'],
    }
]


In [ ]:
runRandomizedSearchCV(linear_svc_rs, [models[0]], [model_names[0]])

Currently training model:  SVM
Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=20 
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=20 
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=20 
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=1.0 


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True",)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=1.0, total=  11.8s
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l1, clf__loss=hinge, clf__C=1.0 


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True",)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=20, total=  12.7s


...

[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=1.0, total=   3.3s
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=1.0 


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError("Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True",)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l1, clf__loss=squared_hinge, clf__C=1.0, total=   3.4s
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__penalty=l2, clf__loss=squared_hinge, clf__C=80 
[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__penalty=l2, clf__loss=hinge, clf__C=80, total=  42.1s
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__penalty=l2, clf__loss=squared_hinge, clf__C=80 
[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__penalty=l2, clf__loss=hinge, clf__C=80, total=  44.1s
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__penalty=l2, clf__loss=squared_hinge, clf__C=80 
[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__penalty=l2, clf__loss=squared_hinge, clf__C=50, total= 1.2min


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.9min


[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__penalty=l2, clf__loss=hinge, clf__C=10 


In [ ]:
runGridSearchCV(linear_svc_pg, [models[0]], [model_names[0]])

### Logistic Regression Parameters

In [46]:
logistic_regression_pg = [
        {
            'vect__ngram_range': [(1,1),(1,2)],
            'tfidf__use_idf': [True,False],
            'clf__C': [0.01, 1.0, 100.],
            'clf__penalty': ['l2','l1','elasticnet'],
            'clf__solver': ['newton-cg','sag','lbfgs','liblinear','saga'],
        }
]

In [55]:
runRandomizedSearchCV(logistic_regression_pg, [models[1]], [model_names[1]])

Currently training model:  Logistic Regression
Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__solver=sag, clf__penalty=elasticnet, clf__C=1.0 
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__solver=sag, clf__penalty=elasticnet, clf__C=1.0 
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__solver=sag, clf__penalty=elasticnet, clf__C=1.0 
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__solver=sag, clf__penalty=l2, clf__C=0.01 


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError('Solver sag supports only l2 penalties, got elasticnet penalty.',)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__solver=sag, clf__penalty=elasticnet, clf__C=1.0, total=   3.0s


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:479: FitFailedWarning: Classifier fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
ValueError('Solver sag supports only l2 penalties, got elasticnet penalty.',)
  "Details: \n%r" % (error_score, e), FitFailedWarning)


[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=False, clf__solver=sag, clf__penalty=elasticnet, clf__C=1.0, total=   3.0s
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__solver=sag, clf__penalty=l2, clf__C=0.01 
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__solver=sag, clf__penalty=l2, clf__C=0.01 


...

/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__solver=saga, clf__penalty=l2, clf__C=100.0, total= 2.1min
[CV] vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__solver=saga, clf__penalty=l2, clf__C=100.0 


/Users/Harsh-Mac/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV]  vect__ngram_range=(1, 2), tfidf__use_idf=True, clf__solver=sag, clf__penalty=l2, clf__C=100.0, total= 8.8min
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__solver=lbfgs, clf__penalty=l2, clf__C=1.0 
[CV]  vect__ngram_range=(1, 1), tfidf__use_idf=True, clf__solver=saga, clf__penalty=l2, clf__C=100.0, total= 2.2min
[CV] vect__ngram_range=(1, 2), tfidf__use_idf=False, clf__solver=lbfgs, clf__penalty=l2, clf__C=1.0 


KeyboardInterrupt: 

In [ ]:
runGridSearchCV(logistic_regression_pg, [models[1]], [model_names[1]])